In [102]:
import sys
import threading

# This is a placeholder for a Google-internal import.

import grpc
import numpy
import tensorflow as tf
import numpy as np

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [103]:
hostport="127.0.0.1:8500"

In [104]:
channel = grpc.insecure_channel(hostport)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [105]:
request = predict_pb2.PredictRequest()
request.model_spec.name = 'bitfinex_btc_90'
request.model_spec.signature_name = 'portfolio_vector'

In [106]:
import json
with open("data.json") as f:
    data = json.load(f)

In [107]:
inp = np.array(data['instances'][0]['input'])
prev_w = np.array(data['instances'][0]['previous_w'])

In [108]:
request.inputs['input'].CopyFrom(tf.contrib.util.make_tensor_proto(inp, shape=[1, inp.size]))
request.inputs['input_num'].CopyFrom(tf.contrib.util.make_tensor_proto(1, shape=[]))
request.inputs['previous_w'].CopyFrom(tf.contrib.util.make_tensor_proto(prev_w,shape=[prev_w.size]))

In [109]:
result_future = stub.Predict(request, 50.0)  # 5 seconds

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Connect Failed"
	debug_error_string = "{"created":"@1537289117.066080412","description":"Failed to create subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":2608,"referenced_errors":[{"created":"@1537289117.066077962","description":"Pick Cancelled","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":234,"referenced_errors":[{"created":"@1537289096.536252437","description":"Connect Failed","file":"src/core/ext/filters/client_channel/subchannel.cc","file_line":663,"grpc_status":14,"referenced_errors":[{"created":"@1537289096.536174928","description":"Failed to connect to remote host: OS Error","errno":111,"file":"src/core/lib/iomgr/tcp_client_posix.cc","file_line":202,"os_error":"Connection refused","syscall":"connect","target_address":"ipv4:127.0.0.1:8500"}]}]}]}"
>